In [370]:
# !pip install spacy
# !python -m spacy download fr_core_news_sm
# !pip3 install emoji
# !pip3 install deep-translator

In [3]:
import nltk
import time
import pandas as pd
from nltk.stem.snowball import FrenchStemmer
import spacy
import argparse
import fr_core_news_sm
from nltk.stem.snowball import SnowballStemmer
import emoji
from deep_translator import GoogleTranslator
import re
from nltk.corpus import words

In [4]:
nlp = spacy.load("fr_core_news_sm")

In [36]:
dataframe_name = 'beauty_wellbeing.csv'

In [37]:
df = pd.read_csv(dataframe_name)

In [38]:
df

,Unnamed: 0,title,comments,stars,time
0,0,\n Bon produit\n ...,\n Bon produit. Livraison rapid...,5 étoiles : excellent,2021-06-19T05:05:28+00:00
1,1,\n Envoi très rapide\n ...,"\n Envoi très rapide, excellent...",5 étoiles : excellent,2021-06-18T14:03:47+00:00
2,2,\n super envoi- on aura...,\n super envoi- on aurait aime ...,5 étoiles : excellent,2021-06-17T23:14:19+00:00
3,3,\n Commande parvenue ra...,\n Commande parvenue rapidement...,5 étoiles : excellent,2021-06-17T17:54:31+00:00
4,4,\n Site sérieux je reco...,\n Très contente de cet achat e...,5 étoiles : excellent,2021-06-17T16:02:08+00:00
...,...,...,...,...,...
12667,12667,\n Je recommande à 100%...,\n Produit de qualité et suivi ...,5 étoiles : excellent,2021-03-04T11:35:50+00:00
12668,12668,\n Produits top... mais...,"\n Produits de qualité, équipe ...",5 étoiles : excellent,2021-03-04T10:19:19+00:00
12669,12669,\n Très satisfaite de l...,\n Première fois que je testais...,5 étoiles : excellent,2021-03-04T09:51:05+00:00
12670,12670,\n Très bien\n ...,"\n Très bien, bons produits\n ...",5 étoiles : excellent,2021-03-04T09:11:45+00:00


In [39]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [40]:
df

,title,comments,stars,time
0,\n Bon produit\n ...,\n Bon produit. Livraison rapid...,5 étoiles : excellent,2021-06-19T05:05:28+00:00
1,\n Envoi très rapide\n ...,"\n Envoi très rapide, excellent...",5 étoiles : excellent,2021-06-18T14:03:47+00:00
2,\n super envoi- on aura...,\n super envoi- on aurait aime ...,5 étoiles : excellent,2021-06-17T23:14:19+00:00
3,\n Commande parvenue ra...,\n Commande parvenue rapidement...,5 étoiles : excellent,2021-06-17T17:54:31+00:00
4,\n Site sérieux je reco...,\n Très contente de cet achat e...,5 étoiles : excellent,2021-06-17T16:02:08+00:00
...,...,...,...,...
12667,\n Je recommande à 100%...,\n Produit de qualité et suivi ...,5 étoiles : excellent,2021-03-04T11:35:50+00:00
12668,\n Produits top... mais...,"\n Produits de qualité, équipe ...",5 étoiles : excellent,2021-03-04T10:19:19+00:00
12669,\n Très satisfaite de l...,\n Première fois que je testais...,5 étoiles : excellent,2021-03-04T09:51:05+00:00
12670,\n Très bien\n ...,"\n Très bien, bons produits\n ...",5 étoiles : excellent,2021-03-04T09:11:45+00:00


In [41]:
def clean_txt(txt: str) -> str:
    
    # Keep alphanumerical
    txt = txt.replace('.', '')
    txt = txt.replace("'", '')
    txt = txt.replace('!', '')
    txt = txt.replace('?', '')
    txt = txt.replace('-', '')
    txt = txt.replace('_', ' ')
    txt = txt.replace('&', '')
    txt = txt.replace(',', '')
    txt = txt.replace(':', '')
    txt = txt.replace('<', '')
    txt = txt.replace('>', '')
    txt = txt.replace('*', '')
    txt = txt.replace('#', '')
    txt = txt.replace('@', '')
    txt = txt.replace(';', '')
    txt = txt.replace('%', '')
    txt = txt.replace('$', '')
    txt = txt.replace('€', '')
    txt = txt.replace('/', '')
    txt = txt.replace('(', '')
    txt = txt.replace(')', '')
    txt = txt.replace('§', '')
    txt = txt.strip().lower()

    txt = str("".join(filter(lambda x: not x.isdigit(), txt)))

    return txt

In [42]:
def translate_en_to_fr(txt: str):
    
    string_txt = " "
    list_words_translated = []
    
    sentence = re.sub(' +', ' ', txt).split(' ')
    
    for word in sentence:
        list_words_translated.append(GoogleTranslator(source='auto', target='fr').translate(word))
    return string_txt.join(list_words_translated)

In [43]:
def text_has_emoji(txt: str):
    
    sentence = " "
    list_emojis = []
    list_words = []
    
    if bool(emoji.get_emoji_regexp().search(txt)) is True:
        for word in txt.split(" "):
            if bool(emoji.get_emoji_regexp().search(word)) is True:
                list_emojis.append(clean_txt(emoji.demojize(word, delimiters=("", ""))))
            else:
                list_words.append(word)

    if len(list_emojis) > 0:
        return sentence.join(list_words) + " " + translate_en_to_fr(sentence.join(list_emojis))
    else:
        return txt
    

In [44]:
def lemmatizer(txt: str):
      
    string_txt = " "
    list_words_lemmat = []
    doc = nlp(txt)

    for val in doc:
        list_words_lemmat.append(val.lemma_)

    return string_txt.join(list_words_lemmat)

In [45]:
# French stop words
list_stopWords = ["a", "à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais", "avons", "avez","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceux-là","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dix-sept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","elles-mêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre-vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut","alors","aucuns","bon","devrait","dos","droite","début","essai","faites","fois","force","haut","ici","juste","maintenant","mine","mot","nommés","nouveaux","parce","parole","personnes","pièce","plupart","seulement","soyez","sujet","tandis","valeur","voie","voient","état","étions"]

In [46]:
def remove_stop_words(txt: str):
    
    txt = lemmatizer(txt)
    
    string_txt = " " 
    list_words = []
    
    sentence = txt.split()

    for val in sentence:
        if val not in list_stopWords:
            list_words.append(val)

    return string_txt.join(list_words)

In [47]:
stemmer = SnowballStemmer(language='french')

In [48]:
def return_stem(txt: str):
    
    txt = remove_stop_words(txt)
    
    string_txt = " "
    list_words = []

    doc = nlp(txt)
    for X in doc:
        list_words.append(stemmer.stem(X.text))
        
    return string_txt.join(list_words)

In [49]:
df_new = df.copy(deep=True)

df_new = df_new.reset_index(drop=True)

In [50]:
start_time = time.time()

In [51]:
df_new['comments'] = df_new['comments'].apply(lambda x: text_has_emoji(x))

In [52]:
df_new['comments'] = df_new['comments'].apply(lambda x: clean_txt(x))

In [53]:
df_new['comments'] = df_new['comments'].apply(lambda x: lemmatizer(x))

In [54]:
df_new['comments'] = df_new['comments'].apply(lambda x: remove_stop_words(x))

In [55]:
# df['comments'] = df['comments'].apply(lambda x: return_stem(x))

In [58]:
df_new.to_csv('cleaned_'+ dataframe_name, index=False)

In [59]:
print((time.time() - start_time)/60, 'minutes')

14.704242849349976 minutes
